# Выбор локации для скважины

Добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Есть пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 
Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 
Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищем месторождения, для каждого определяем значения признаков;
- Строим модель и оцениваем объём запасов;
- Выбираем месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание данных

Данные геологоразведки трёх регионов находятся в 3 файлах.


- id — уникальный идентификатор скважины;

- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

- product — объём запасов в скважине (тыс. баррелей).


Условия задачи:

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

- Бюджет на разработку скважин в регионе — 10 млрд рублей.

- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

### Загрузка данных

In [1]:
#импорт библиотек
import pandas as pd

import numpy as np
from numpy.random import RandomState

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
try: # чтение файла с сайта
    geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except: # чтение файла локально
    geo_data_0 = pd.read_csv('datasets/geo_data_0.csv')
    geo_data_1 = pd.read_csv('datasets/geo_data_1.csv')
    geo_data_2 = pd.read_csv('datasets/geo_data_2.csv')

Обзор данных:

In [3]:
display(geo_data_0.head())
display(geo_data_1.head())
display(geo_data_2.head())
print('----------------------------')
print('Размер:')
print(geo_data_0.shape)
print(geo_data_1.shape)
print(geo_data_2.shape)
print('----------------------------')
print('Информация:')
print(geo_data_0.info())
print(geo_data_1.info())
print(geo_data_2.info())
print('----------------------------')
print('Дубликаты:')
print(geo_data_0.duplicated().sum())
print(geo_data_1.duplicated().sum())
print(geo_data_2.duplicated().sum())
print('----------------------------')
print('Описание данных:')
display(geo_data_0.describe())
display(geo_data_1.describe())
display(geo_data_2.describe())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


----------------------------
Размер:
(100000, 5)
(100000, 5)
(100000, 5)
----------------------------
Информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class '

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Размеры датасетов одинаковые.

Дубликатов нет.

Пропусков нет.

Параметры f0, f1, f2 имеют разные минимумы, максимумы, средние. Необходимо будет привести к единому масштабу.

Названия столбцов соответствуют описанию.

Название скважины не помогает обучению модели. Уберём этот столбец.



In [4]:
geo_data_0 = geo_data_0.drop('id', axis=1) 
geo_data_1 = geo_data_1.drop('id', axis=1) 
geo_data_2 = geo_data_2.drop('id', axis=1) 

display(geo_data_0.head(3)) # проверка
display(geo_data_1.head(3))
display(geo_data_2.head(3))

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910


### Разделение данных на выборки, извлечение признаков.

Функция для разделения выборок и извлечения признаков:

In [5]:
def target_n_features (geo_data):
    target = geo_data['product']                                     # целевой признак
    features = geo_data.drop(['product'] , axis=1)                   # признаки
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)            # валидационная выборка - 25%
    return(features_train, features_valid, target_train, target_valid)

Данные первого региона:

In [6]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = target_n_features(geo_data_0)
print(features_train_0.shape, target_train_0.shape)                  # контроль размера выборок
print(features_valid_0.shape, target_valid_0.shape)

(75000, 3) (75000,)
(25000, 3) (25000,)


Данные второго региона:

In [7]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = target_n_features(geo_data_1)
print(features_train_1.shape, target_train_1.shape)                   # контроль размера выборок
print(features_valid_1.shape, target_valid_1.shape)

(75000, 3) (75000,)
(25000, 3) (25000,)


Данные третьего региона:

In [8]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = target_n_features(geo_data_2)
print(features_train_2.shape, target_train_2.shape)                  # контроль размера выборок
print(features_valid_2.shape, target_valid_2.shape)

(75000, 3) (75000,)
(25000, 3) (25000,)


### Подготовка данных.


Приведём признаки к единому масштабу.
Используем метод стандартизации данных. 

In [9]:
scaler = StandardScaler()
# первый регион
scaler.fit(features_train_0)
features_train_0 = scaler.transform(features_train_0)
features_valid_0 = scaler.transform(features_valid_0)
# второй регион
scaler.fit(features_train_1) 
features_train_1 = scaler.transform(features_train_1)
features_valid_1 = scaler.transform(features_valid_1)
# третий регион
scaler.fit(features_train_2) 
features_train_2 = scaler.transform(features_train_2)
features_valid_2 = scaler.transform(features_valid_2)

## Обучение и проверка модели

Напишем функцию, в которой применяется модель линейной регрессии для предсказания запасов в регионе.

In [10]:
def model_ (features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = pd.Series(model.predict(features_valid), index = target_valid.index)
    mse = mean_squared_error(target_valid, predicted_valid)
    print("Средний запас предсказанного сырья =", '%.2f' % predicted_valid.mean(), 'тыс. баррелей')
    print("RMSE модели линейной регрессии =", '%.2f' % mse ** 0.5)
    return predicted_valid

### Регион 1

Работа модели:

In [11]:
predicted_valid_0 = model_(features_train_0, target_train_0, features_valid_0, target_valid_0)

Средний запас предсказанного сырья = 92.59 тыс. баррелей
RMSE модели линейной регрессии = 37.58


Качество предсказания низкое, среднеквадратическая почти 50% от среднего. 

Сравним с работой константной модели:

In [12]:
mean_valid_0 = pd.Series(target_train_0.mean(), index=target_valid_0.index) 
mse = mean_squared_error(target_valid_0, mean_valid_0)
print("RMSE константной модели =", '%.2f' % mse ** 0.5)

RMSE константной модели = 44.29


Качество работы модели линейной регрессии выше, модель проходит проверку на адекватность.

### Регион 2

Работа модели:

In [13]:
predicted_valid_1 = model_(features_train_1, target_train_1, features_valid_1, target_valid_1)

Средний запас предсказанного сырья = 68.73 тыс. баррелей
RMSE модели линейной регрессии = 0.89


Качество предсказания высокое, среднеквадратическая ошибка 1,27 % от среднего. 

Сравним с работой константной модели:

In [14]:
mean_valid_1 = pd.Series(target_train_1.mean(), index=target_valid_1.index)
mse = mean_squared_error(target_valid_1, mean_valid_1)
print("RMSE константной модели =", '%.2f' % mse ** 0.5)

RMSE константной модели = 46.02


Качество работы модели линейной регрессии выше, модель проходит проверку на адекватность.

### Регион 3

Работа модели:

In [15]:
predicted_valid_2 = model_(features_train_2, target_train_2, features_valid_2, target_valid_2)

Средний запас предсказанного сырья = 94.97 тыс. баррелей
RMSE модели линейной регрессии = 40.03


Качество предсказания низкое, почти 50% от среднего. 

Сравним с работой константной модели:

In [16]:
mean_valid_2 = pd.Series(target_train_2.mean(), index=target_valid_2.index)
mse = mean_squared_error(target_valid_2, mean_valid_2)
print("RMSE константной модели =", '%.2f' % mse ** 0.5)

RMSE константной модели = 44.90


Качество работы модели линейной регрессии выше, модель проходит проверку на адекватность.

### Вывод: 
Самое качественное предсказание модели во втором регионе.

## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

Бюджет на разработку скважин в регионе — 10 млрд рублей.

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. 

Среди них выберем регион с наибольшей средней прибылью.

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

Запишем числа в переменные-константы.

In [17]:
INVEST = 10000000000            # инвестиции в регион
NUM_WELLS_EXPLORE = 500         # количество исследуемых точек в регионе
NUM_WELLS_CHOSEN = 200          # количество разрабатываемых точек в регионе
WELL_INCOME = 450000            # доход с 1 скважины


На одну скважину приходится бюджет:

In [18]:
print(INVEST/NUM_WELLS_CHOSEN, 'руб')

50000000.0 руб


Необходимый объём сырья:

In [19]:
print((INVEST/NUM_WELLS_CHOSEN)/WELL_INCOME, 'тыс.баррелей')

111.11111111111111 тыс.баррелей


Средний реальный запас в регионах:

In [20]:
print('В первом регионе:', '%.2f' % geo_data_0['product'].mean())
print('Во втором регионе:', '%.2f' % geo_data_1['product'].mean())
print('В третьем регионе:', '%.2f' % geo_data_2['product'].mean())

В первом регионе: 92.50
Во втором регионе: 68.83
В третьем регионе: 95.00


Средний предсказанный запас в регионах:

In [21]:
print('В первом регионе:', '%.2f' % predicted_valid_0.mean())
print('Во втором регионе:', '%.2f' % predicted_valid_1.mean())
print('В третьем регионе:', '%.2f' % predicted_valid_2.mean())

В первом регионе: 92.59
Во втором регионе: 68.73
В третьем регионе: 94.97


### Вывод: 
Средний запас слишком мал для безубыточной разработки всех найденных скважин. Необходимо проводить предварительное исследование и отбор скважин, чтобы найти самые прибыльные.

## Расчёт прибыли и рисков 

### Расчёт прибыли

Напишем функцию, вычисляющую прибыль из региона. Такая прибыль может быть достигнута, если исследовать не 500, а все 10 000 точек.

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [22]:
def revenue (predicted, target):
    indexes = predicted.sort_values(ascending=False)[0:NUM_WELLS_CHOSEN].index  # индексы скважин с лучшими предсказаниями
    revenue_per_one = target[indexes]*WELL_INCOME                               # подсчёт реальной прибыли с этих скважин
    return revenue_per_one.sum()-INVEST

Предсказанная максимальная прибыль:

In [23]:
print('Прибыль первого региона:', round(revenue(predicted_valid_0, target_valid_0)/1000000, 3), 'млн.руб.')
print('Прибыль второго региона:', round(revenue(predicted_valid_1, target_valid_1)/1000000, 3), 'млн.руб.')
print('Прибыль третьего региона:', round(revenue(predicted_valid_2, target_valid_2)/1000000, 3), 'млн.руб.')

Прибыль первого региона: 3320.826 млн.руб.
Прибыль второго региона: 2415.087 млн.руб.
Прибыль третьего региона: 2710.35 млн.руб.


### Расчёт рисков и прибылей 


Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [24]:
def risk(predicted, target):
    state = np.random.RandomState(12345)
    values = []   # массив возможных прибылей
    for i in range(1000):
        predicted_subsample = predicted.sample(
            frac=NUM_WELLS_EXPLORE/len(predicted),         # в выборку попадёт количество исследуемых скважин
            replace=True, random_state=state)
        tagret_subsample = target[predicted_subsample.index]
        values.append(revenue(predicted_subsample, tagret_subsample))  # добавляем вычисленную прибыль по выборке
    values = pd.Series(values)
    higher = values.quantile(.975)                         # с вероятностью 97,5% прибыль будет ниже этого значения
    lower = values.quantile(.025)                          # с вероятностью 2,5% прибыль будет ниже этого значения
    mean = values.mean()  # средняя прибыль
    print("Средняя прибыль:", round(mean/1000000, 3), 'млн.руб.')
    print("97.5%-квантиль:", round(higher/1000000, 3), 'млн.руб.')
    print("2,5%-квантиль:", round(lower/1000000, 3), 'млн.руб.')
    print('Минимальная прибыль:', round(values.min()/1000000, 3), 'млн.руб.')
    print('Вероятность убытков:', len(values.loc[values<0])/len(values)*100, "%")

Рассчитаем среднюю прибыль, 95%-й доверительный интервал и риск убытков для первого региона:

In [25]:
risk(predicted_valid_0, target_valid_0)

Средняя прибыль: 600.735 млн.руб.
97.5%-квантиль: 1231.164 млн.руб.
2,5%-квантиль: 12.948 млн.руб.
Минимальная прибыль: -264.535 млн.руб.
Вероятность убытков: 2.0 %


Рассчитаем среднюю прибыль, 95%-й доверительный интервал и риск убытков для второго региона:

In [26]:
risk(predicted_valid_1, target_valid_1)

Средняя прибыль: 665.241 млн.руб.
97.5%-квантиль: 1197.642 млн.руб.
2,5%-квантиль: 157.988 млн.руб.
Минимальная прибыль: -105.935 млн.руб.
Вероятность убытков: 0.3 %


Рассчитаем среднюю прибыль, 95%-й доверительный интервал и риск убытков для третьего региона:

In [27]:
risk(predicted_valid_2, target_valid_2)

Средняя прибыль: 615.56 млн.руб.
97.5%-квантиль: 1230.644 млн.руб.
2,5%-квантиль: -12.218 млн.руб.
Минимальная прибыль: -397.674 млн.руб.
Вероятность убытков: 3.0 %


### Вывод:
Регион 3 показывает убыток с вероятностью 3%, что выше порогового значения в 2,5%. Следовательно, третий регион не подходит по условию риска. 

Наибольшая средняя прибыль рассчитана во втором регионе и составляет 665.241 млн.руб. 

Другие преимущества второго региона - самая точная модель (1,27% ошибки) и самый низкий риск убытка (0,3%).

По итогам исследования для разработки предлагается второй регион.